## Vérification et nettoyage des données - Projet 6

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm;
from datetime import datetime

%matplotlib inline

folder = "/Users/yathyarayamauchiferreira/Documents/OpenClassrooms/Projet 6/"
customers = folder + "customers.csv"
products = folder + "products.csv"
transactions = folder + "transactions.csv"

#### Lecture du fichier CUSTOMERS

In [2]:
df_customers = pd.read_csv(customers)
df_customers.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [16]:
df_customers[df_customers.client_id.isin(['c_1609','c_4958','c_6714','c_3454'])]

,client_id,sex,birth
1378,c_4958,m,1999
1911,c_6714,f,1968
3642,c_1609,m,1980
8088,c_3454,m,1969


In [6]:
df_customers.shape

(8623, 3)

In [7]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [8]:
df_customers.describe()

,birth
count,8623.000000
mean,1978.280877
std,16.919535
min,1929.000000
25%,1966.000000
50%,1979.000000
75%,1992.000000
max,2004.000000


In [15]:
df_customers.client_id.duplicated().sum()

0

In [14]:
df_customers[df_customers["client_id"].str.contains("ct")]

,client_id,sex,birth
2735,ct_0,f,2001
8494,ct_1,m,2001


**Suppression des clients tests:**

In [9]:
df_customers.drop(df_customers[(df_customers["client_id"] == "ct_0")].index, inplace=True)
df_customers.drop(df_customers[(df_customers["client_id"] == "ct_1")].index, inplace=True)
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8621 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8621 non-null   object
 1   sex        8621 non-null   object
 2   birth      8621 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 269.4+ KB


**Ajout d'une colonne "age" aux clients:**

In [10]:
current_year = datetime.now().strftime('%Y')
df_customers["age"] = int(current_year) - df_customers["birth"]
df_customers.head()

,client_id,sex,birth,age
0,c_4410,f,1967,56
1,c_7839,f,1975,48
2,c_1699,f,1984,39
3,c_5961,f,1962,61
4,c_5320,m,1943,80


#### Lecture du fichier PRODUCTS

In [8]:
df_products = pd.read_csv(products)
df_products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [9]:
df_products.shape

(3287, 3)

In [10]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


In [11]:
df_products.describe()

,price,categ
count,3287.000000,3287.000000
mean,21.856641,0.370246
std,29.847908,0.615387
min,-1.000000,0.000000
25%,6.990000,0.000000
50%,13.060000,0.000000
75%,22.990000,1.000000
max,300.000000,2.000000


Le prix -1 correspond à un produit identifié comme T_0, on suppose qu'il s'agit d'un produit test qui pourrait être supprimé (01 ligne identifiée)

In [12]:
df_products[(df_products["id_prod"] == "T_0")]

,id_prod,price,categ
731,T_0,-1.0,0


**Suppression du produit test:**

In [13]:
df_products.drop(df_products[(df_products["id_prod"] == "T_0")].index, inplace=True)
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3286 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3286 non-null   object 
 1   price    3286 non-null   float64
 2   categ    3286 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 102.7+ KB


#### Lecture du fichier TRANSACTIONS

In [16]:
df_transactions = pd.read_csv(transactions)
df_transactions.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


In [17]:
df_transactions.shape

(679532, 4)

In [18]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679532 entries, 0 to 679531
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     679532 non-null  object
 1   date        679532 non-null  object
 2   session_id  679532 non-null  object
 3   client_id   679532 non-null  object
dtypes: object(4)
memory usage: 20.7+ MB


In [19]:
df_transactions.describe()

,id_prod,date,session_id,client_id
count,679532,679532,679532,679532
unique,3267,679371,342316,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,2252,13,200,25488


In [20]:
df_transactions[df_transactions.date.str.contains('^test')]

,id_prod,date,session_id,client_id
3019,T_0,test_2021-03-01 02:30:02.237419,s_0,ct_0
5138,T_0,test_2021-03-01 02:30:02.237425,s_0,ct_0
9668,T_0,test_2021-03-01 02:30:02.237437,s_0,ct_1
10728,T_0,test_2021-03-01 02:30:02.237436,s_0,ct_0
15292,T_0,test_2021-03-01 02:30:02.237430,s_0,ct_0
...,...,...,...,...
657830,T_0,test_2021-03-01 02:30:02.237417,s_0,ct_0
662081,T_0,test_2021-03-01 02:30:02.237427,s_0,ct_1
670680,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_1
671647,T_0,test_2021-03-01 02:30:02.237424,s_0,ct_1


**200 lignes test identifiées**

**Suppression du produit tests:**

In [21]:
df_transactions.drop(df_transactions[df_transactions.date.str.contains('^test')].index,inplace=True)
df_transactions[df_transactions.date.str.contains('^test')]

,id_prod,date,session_id,client_id


**Transformer la colonne date en datetime:**

In [60]:
df_transactions["date"] = pd.to_datetime(df_transactions["date"], yearfirst= True)
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679531
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  object        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  object        
 3   client_id   679332 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 25.9+ MB


In [61]:
df_transactions.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


**Jointure des dataframes:** 

Première jointure:

In [62]:
df_trans_custom = pd.merge(df_transactions, df_customers, on = "client_id", how = "left")
df_trans_custom.head()

,id_prod,date,session_id,client_id,sex,birth,age
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,f,1986,37
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,m,1988,35
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,f,1968,55
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,m,2000,23
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,m,1980,43


In [63]:
df_trans_custom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679331
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  object        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  object        
 3   client_id   679332 non-null  object        
 4   sex         679332 non-null  object        
 5   birth       679332 non-null  int64         
 6   age         679332 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 41.5+ MB


le df transaction garde le même nombre de ligne 

Seconde jointure:

In [64]:
df_complet = pd.merge(df_trans_custom, df_products, on = "id_prod", how = "left")
df_complet.head()

,id_prod,date,session_id,client_id,sex,birth,age,price,categ
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,f,1986,37,4.18,0.0
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,m,1988,35,15.99,1.0
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,f,1968,55,7.99,0.0
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,m,2000,23,69.99,2.0
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,m,1980,43,4.99,0.0


In [65]:
df_complet.isna().sum()

id_prod         0
date            0
session_id      0
client_id       0
sex             0
birth           0
age             0
price         221
categ         221
dtype: int64

In [66]:
maskna = df_complet.price.isna()
df_complet[maskna]

,id_prod,date,session_id,client_id,sex,birth,age,price,categ
2633,0_2245,2022-09-23 07:22:38.636773,s_272266,c_4746,m,1940,83,NaN,NaN
10103,0_2245,2022-07-23 09:24:14.133889,s_242482,c_6713,f,1963,60,NaN,NaN
11723,0_2245,2022-12-03 03:26:35.696673,s_306338,c_5108,m,1978,45,NaN,NaN
15670,0_2245,2021-08-16 11:33:25.481411,s_76493,c_1391,m,1991,32,NaN,NaN
16372,0_2245,2022-07-16 05:53:01.627491,s_239078,c_7954,m,1973,50,NaN,NaN
...,...,...,...,...,...,...,...,...,...
669533,0_2245,2021-08-25 09:06:03.504061,s_80395,c_131,m,1981,42,NaN,NaN
670484,0_2245,2022-03-06 19:59:19.462288,s_175311,c_4167,f,1979,44,NaN,NaN
671088,0_2245,2022-05-16 11:35:20.319501,s_209381,c_4453,m,1981,42,NaN,NaN
675480,0_2245,2022-02-11 09:05:43.952857,s_163405,c_1098,m,1986,37,NaN,NaN


In [67]:
df_complet[maskna].id_prod.value_counts()

0_2245    221
Name: id_prod, dtype: int64

Le produit 0_2245 n'existe pas sur le fichier PRODUCTS et le prix est NaN (catégorie 0)

On remplace le prix NaN par zéro ou par la moyenne des autres prix de la catégorie 0:

In [72]:
df_complet["categ"] = np.where(df_complet["id_prod"] == "0_2245",
                               0, df_complet["categ"])
df_complet["price"] = np.where(df_complet["id_prod"] == "0_2245",
                               df_complet.loc[(df_complet["categ"]==0),"price"].median(),
                               df_complet["price"])
df_complet[df_complet["id_prod"] == "0_2245"]


,id_prod,date,session_id,client_id,sex,birth,age,price,categ,year,month,day
2633,0_2245,2022-09-23 07:22:38.636773,s_272266,c_4746,m,1940,83,9.99,0.0,2022,9,23
10103,0_2245,2022-07-23 09:24:14.133889,s_242482,c_6713,f,1963,60,9.99,0.0,2022,7,23
11723,0_2245,2022-12-03 03:26:35.696673,s_306338,c_5108,m,1978,45,9.99,0.0,2022,12,3
15670,0_2245,2021-08-16 11:33:25.481411,s_76493,c_1391,m,1991,32,9.99,0.0,2021,8,16
16372,0_2245,2022-07-16 05:53:01.627491,s_239078,c_7954,m,1973,50,9.99,0.0,2022,7,16
...,...,...,...,...,...,...,...,...,...,...,...,...
669533,0_2245,2021-08-25 09:06:03.504061,s_80395,c_131,m,1981,42,9.99,0.0,2021,8,25
670484,0_2245,2022-03-06 19:59:19.462288,s_175311,c_4167,f,1979,44,9.99,0.0,2022,3,6
671088,0_2245,2022-05-16 11:35:20.319501,s_209381,c_4453,m,1981,42,9.99,0.0,2022,5,16
675480,0_2245,2022-02-11 09:05:43.952857,s_163405,c_1098,m,1986,37,9.99,0.0,2022,2,11


**Création des nouvelles colonnes pour séparer des dates et faciliter la manipulation:**

In [75]:
df_complet["year"] = df_complet["date"].dt.year
df_complet['month'] = df_complet['date'].dt.strftime('%Y-%m')
df_complet['day'] = df_complet['date'].dt.strftime('%Y-%m-%d')
df_complet.info()
df_complet.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679331
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  object        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  object        
 3   client_id   679332 non-null  object        
 4   sex         679332 non-null  object        
 5   birth       679332 non-null  int64         
 6   age         679332 non-null  int64         
 7   price       679332 non-null  float64       
 8   categ       679332 non-null  float64       
 9   year        679332 non-null  int64         
 10  month       679332 non-null  object        
 11  day         679332 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 67.4+ MB


,id_prod,date,session_id,client_id,sex,birth,age,price,categ,year,month,day
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,f,1986,37,4.18,0.0,2022,2022-05,2022-05-20
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,m,1988,35,15.99,1.0,2022,2022-02,2022-02-02
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,f,1968,55,7.99,0.0,2022,2022-06,2022-06-18
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,m,2000,23,69.99,2.0,2021,2021-06,2021-06-24
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,m,1980,43,4.99,0.0,2023,2023-01,2023-01-11


#### générer le fichier df_complet CSV traité

In [76]:
folder = "/Users/yathyarayamauchiferreira/Documents/OpenClassrooms/Projet 6/"
df_complet.to_csv(folder + 'df_complet.csv', index=False)